# 0. Import required packages

In [ ]:
import os
import yaml
import math
import numpy as np
import pandas as pd
from scipy import constants as spcon
from matplotlib import pyplot as plt
from reportlab.lib import colors
import tools

In [ ]:
from massibo_ana.postprocess.PDFGenerator import PDFGenerator
import massibo_ana.utils.custom_exceptions as cuex

# 1. Load the input parameters and change the WD to the workspace

In [ ]:
# Write here the ABSOLUTE path to the input parameters file
input_parameters_filepath = ''

with open(
        input_parameters_filepath,
        "r",
        encoding="utf-8"
    ) as file:
        
        params = yaml.safe_load(file)

os.chdir(params['workspace_dir'])
load_dir = "load/"
summary_dir = "summary/"

# 2. Read the waveforms, rebin, compute baseline and flip

In [ ]:
gainmeas_objects = tools.build_list_of_SiPMMeas_objects(
    load_dir,
    params['strips_to_analyze'],
    params['analyzable_marks'],
    is_gain_meas=True,
    verbose=params['verbose']
)

if len(gainmeas_objects) == 0:
    raise Exception(
        "Not a single GainMeas object was found coming from "
        f"a board with a strip ID within {params['strips_to_analyze']} "
        f"and with a mark within {params['analyzable_marks']}."
    )

analysis_reliability = [2] * len(gainmeas_objects)

for i, gno in enumerate(gainmeas_objects):

    gno.rebin(params['merged_points'])

    if len(gno.Waveforms) < params['minimum_number_of_waveforms']:
        analysis_reliability[i] = 0

    # It is important to compute a reliable baseline before integrating,
    # because the baseline is subtracted from the signal prior to integration.
    gno.Waveforms.compute_first_peak_baseline(
        signal_fraction_for_median_cutoff=params['signal_fraction_for_median_cutoff'],
        half_width_about_median=params['half_width_about_median'],
    )

    if params['flip_about_baseline']:
        gno.Waveforms.flip_about_baseline()

# Order the waveform sets according to their strip IDs
gainmeas_objects = tools.order_list_of_SiPMMeas_objects(gainmeas_objects)

In [ ]:
if len(set([gno.Overvoltage_V for gno in gainmeas_objects])) > 3:
    raise Exception(
        "There are more than 3 different overvoltage "
        "values. This is not supported."
    )

# 3. Start the PDF report

In [ ]:
if params['generate_report']:

    pdf_chunks_filepaths_generator = PDFGenerator.PDF_chunk_filepath(
        summary_dir,
        params['report_output_filename']
    )
    
    aux = next(pdf_chunks_filepaths_generator)
    pdf_reports_filepaths = [aux]
    report_pdf = PDFGenerator(aux)

# 4. Run the analysis

In [ ]:
for idx in range(len(gainmeas_objects)):

    if params['verbose']:
        print(
            f"Integrating the ({len(gainmeas_objects[idx].Waveforms)}) "
            f"waveforms for the {idx+1}-th "
            f"(/{len(gainmeas_objects)}) SiPM data - "
            f"{gainmeas_objects[idx].get_title()}"
        )

    integration_lower_lim_s = \
        gainmeas_objects[idx].Waveforms.find_mean_beginning_of_raise(
            signal_fraction_for_median_cutoff=params['signal_fraction_for_median_cutoff'],
            tolerance=params['rising_edge_tolerance'],
            # Return a time value, which we assume it is in seconds
            return_iterator=False
        ) + params['integration_lower_lim_correction_s']

    integration_upper_lim_s = integration_lower_lim_s + params['integration_lim_width_s']

    # WARNING: Note that here you are setting the input resistance of the DAQ and the
    # amplification factor of the readout system. Check help(GainMeas.integrate).
    gainmeas_objects[idx].integrate(
        input_resistance_in_ohms=params['input_resistance_in_ohms'],
        system_amplification_factor=params['system_amplification_factor'],
        integration_lower_lim=integration_lower_lim_s,
        integration_upper_lim=integration_upper_lim_s
    )

# 5. Fit the gain

In [ ]:
# For every gain measurement, the gaussian fits are done for every
# element in a bidimensional grid of (bins_no x peaks_to_detect).
# Take the fit with the lowest error, we are then identifying the
# fit with the lowest error figure-of-merit. You can check the
# details of such computation in the error_fom variable below.
# Note that the errors coming from the gaussian fits are normalized
# to the number of fitted peaks. Since we are comparing errors among
# fits with different number of fitted peaks, the error should be
# computed on a peak basis, so that fits with less peaks are not
# systematically favored.

errors = {}

# First scan which is 'quicker' because we are not producing plots, i.e. we are
# setting gain_fit_axes and histogram_fit_axes to None in the fit_gain method.
for i in range(len(gainmeas_objects)):

    if params['verbose']:
        print(
            f"Searching for an optimal gain-fit for the "
            f"{i+1}-th (/{len(gainmeas_objects)}) SiPM "
            f"data - {gainmeas_objects[i].get_title()}"
        )

    errors[i] = {}
    for peaks_to_detect in reversed(
        range(
            params['min_peaks_to_detect'],
            params['max_peaks_to_detect']+1
        )
    ):

        errors[i][peaks_to_detect] = {}
        for bins_no in np.linspace(
            params['min_bins_no'],
            params['max_bins_no'],
            params['bins_no_points']
        ):

            errors[i][peaks_to_detect][int(bins_no)] = {}
            try:
                gain_popt, gain_stderr, histogram_popt, histogram_pcov = \
                    gainmeas_objects[i].fit_gain(
                        has_pedestal=True,
                        peaks_to_detect=peaks_to_detect,
                        peaks_to_fit=None,
                        bins_no=int(bins_no),
                        # Should be small enough so that the sampling is fine enough
                        # for the (mean-gaussian_fit_std_no*std, mean+gaussian_fit_std_no*std)
                        # interval to contain enough points for each fit (each gaussian
                        # fit has three free parameters), but big enough so that the
                        # binning is coarse enough for the previous-peak-detection not
                        # to find more than one peak per actual-charge-peak.
                        starting_fraction=params['peak_search_starting_fraction'],
                        step_fraction=params['peak_search_step_fraction'],
                        minimal_prominence_wrt_max=params['minimal_prominence_wrt_max'],
                        std_no=params['gaussian_fit_std_no'],
                        gain_fit_axes=None,
                        histogram_fit_axes=None
                )

                if params['verbose']:
                    print(
                        "Found the requested number of peaks "
                        f"({peaks_to_detect}) for the {i+1}-th "
                        f"(/{len(gainmeas_objects)}) measurement "
                        f"({gainmeas_objects[i].get_title()}) "
                        f"with peaks_to_detect={peaks_to_detect} and "
                        f"bins_no={bins_no}."
                    )
                
            except cuex.RequestedPeaksNotFound:
                if params['verbose']:
                    print(
                        "Couldn't find the requested number of peaks "
                        f"({peaks_to_detect}) for the {i+1}-th "
                        f"(/{len(gainmeas_objects)}) measurement "
                        f"({gainmeas_objects[i].get_title()}) "
                        f"with peaks_to_detect={peaks_to_detect} and "
                        f"bins_no={bins_no}. A different (peaks_to_detect, "
                        "bins_no) combination will be tried."
                    )
                continue

            except RuntimeError as e:
                if params['verbose']:
                    print(
                        "Encountered the following RunTime error while "
                        f"fitting the {i+1}-th (/{len(gainmeas_objects)}) "
                        f"measurement ({gainmeas_objects[i].get_title()}) "
                        f"with peaks_to_detect={peaks_to_detect} "
                        f"and bins_no={bins_no}: \n\t -> {e} \n A different "
                        "(peaks_to_detect, bins_no) combination will be "
                        "tried."
                    )
                    continue

            gaussian_fit_error = 0.0
            for cov_matrix in histogram_pcov:
                gaussian_fit_error += np.sum(np.sqrt(np.diag(cov_matrix)))

            # If any of the numbers involved in the following computation is np.Inf,
            # it will be understood as an infinite error, and it won't become the
            # optimal error. Yes, np.Inf is well-defined under addition,
            # multiplication and comparison.
            errors[i][peaks_to_detect][int(bins_no)]['gaussian_fit_error'] = \
                gaussian_fit_error/len(histogram_pcov)
            errors[i][peaks_to_detect][int(bins_no)]['linear_fit_error'] = \
                gain_stderr[0]+gain_stderr[1]

In [ ]:
# Find the mean and standard deviation of the linear and gaussian
# fit errors. This step is necessary to later normalize both errors
# so that comparison among both is meaningful.
linear_fit_error_samples = {}
gaussian_fit_error_samples = {}

for i in errors.keys():
    # linear_fit_error_samples[i] is a list of linear-fit error
    # samples for the i-th SiPM data. The same data structure is
    # used for gaussian_fit_error_samples[i]
    linear_fit_error_samples[i] = []
    gaussian_fit_error_samples[i] = []

    for peaks_to_detect in errors[i].keys():
        for bins_no in errors[i][peaks_to_detect].keys():
            try:
                # In the previous code, the linear fit error and the gaussian
                # fit error are added to the errors dictionary at the same time.
                # They are mutually defined, i.e. if one is not defined, the other
                # one is not defined either. Therefore, we can safely use the
                # following try-except block for both errors at the same time.
                linear_fit_error_samples[i].append(
                    errors[i][peaks_to_detect][bins_no]['linear_fit_error']
                )
                gaussian_fit_error_samples[i].append(
                    errors[i][peaks_to_detect][bins_no]['gaussian_fit_error']
                )
            except KeyError:
                pass


for i in linear_fit_error_samples.keys():
# Linear fit errors normally contain some zeros, which come from
# situations where only two peaks were fit. We shall not count these in.
    linear_fit_error_samples[i] = np.array(
        tools.filter_out_zeros_and_infs(
            linear_fit_error_samples[i]
        )
    )

    # Gaussian fit errors normally contain some
    # infinite entries. We shall not count these in.
    gaussian_fit_error_samples[i] = np.array(
        tools.filter_out_zeros_and_infs(
            gaussian_fit_error_samples[i]
        )
    )

    # If after filtering out the infinities, there are
    # no gaussian-fit-error samples left, then we tag
    # the analysis for this GainMeas as 1-reliable
    if len(gaussian_fit_error_samples[i]) == 0:
        analysis_reliability[i] = 1

        if params['verbose']:
            print(
                "WARNING: Not a single (peaks_to_detect, bins_no) "
                "combination allowed a finite-error gaussian "
                f"peaks-fit of the charge histogram for the {i+1}-th "
                f"(/{len(gainmeas_objects)}) SiPM data - "
                f"{gainmeas_objects[i].get_title()}. The analysis for "
                "this GainMeas has been tagged as 1-reliable."
            )
        
linear_fit_error_mean = {}
linear_fit_error_std = {}
gaussian_fit_error_mean = {}
gaussian_fit_error_std = {}

for i in errors.keys():
    if analysis_reliability[i] > 1:
        # linear_fit_error_mean[i] and linear_fit_error_std[i] may be NaN
        # if there are no samples left after filtering out the null values
        # associated to the 2-peaks fits.
        linear_fit_error_mean[i] = np.mean(linear_fit_error_samples[i])
        linear_fit_error_std[i] = np.std(linear_fit_error_samples[i])
        # gaussian_fit_error_mean[i] is always defined, because we made
        # sure that that 2-reliable analyses have at least one finite
        # gaussian fit
        gaussian_fit_error_mean[i] = np.mean(gaussian_fit_error_samples[i])
        # gaussian_fit_error_std[i] may be null if only one
        # (peaks_to_detect, bins_no) combination was found to give a finite
        # gaussian fit error
        gaussian_fit_error_std[i] = np.std(gaussian_fit_error_samples[i])

In [ ]:
# For every gain measurement, scan through all of the (peaks_to_detect, bins_no)
# combinations to find the one with the smallest error. To compute a meaningful 
# error figure-of-merit which combines both the linear and gaussian fit errors,
# normalize each contribution by the mean and standard deviation of the corresponding
# distribution.
optimal_parameters = {
    i: {
        'peaks_to_detect': None,
        'bins_no': None,
        'error_fom': None
    } for i in range(len(gainmeas_objects))
}

for i in range(len(gainmeas_objects)):

    if analysis_reliability[i] > 1:

        fConsiderLinearFitError = False
        # If all of the linear fits for this SiPM data 
        # considered just two peaks, then all of the
        # linear-fit error entries for this SiPM data
        # are zero, and they were filtered out. Therefore,
        # np.mean() and np.std() should have resulted in
        # NaN. If that is the case, do not consider the
        # linear fit error in the error figure-of-merit.
        if not math.isnan(linear_fit_error_mean[i]) and \
            not math.isnan(linear_fit_error_std[i]):
            fConsiderLinearFitError = True

        fGaussianFitErrorStdIsZero = True if \
            gaussian_fit_error_std[i] == 0. else False

        for peaks_to_detect in errors[i].keys():
            for bins_no in errors[i][peaks_to_detect].keys():
                try:
                    # N.B. 1: The if-case happens if only 2 peaks were fitted. In
                    # this case, compensate by taking the mean of the normalized
                    # distribution (i.e. zero), so that 2-peaks fits are not
                    # systematically favored. In the else-case, (the computed linear
                    # fit error is not null), normalize the error.
                    # N.B. 2: It may happen that some of the (peaks_to_detect, bins_no)
                    # combinations worked out for this SiPM data, thus giving a
                    # True value for the fConsiderLinearFitError flag, but that the
                    # current (peaks_to_detect, bins_no) combination only fitted two
                    # peaks, thus giving a linear fit error of zero. This is the
                    # exact case we are managing with the if-else below.
                    if fConsiderLinearFitError:
                        linear_fit_error = 0.0 \
                        if errors[i][peaks_to_detect][bins_no]['linear_fit_error'] == 0. \
                        else (errors[i][peaks_to_detect][bins_no]['linear_fit_error'] - \
                            linear_fit_error_mean[i])/linear_fit_error_std[i]

                    # Note that this block of code is executed only if the
                    # analysis reliability is greater than 1, i.e. there was
                    # at least one gaussian fit with a finite error. This
                    # ensures that the gaussian_fit_error_mean is well-defined.
                    # However, the gaussian_fit_error_std might still be zero if
                    # only (exactly) one gaussian fit resulted to have a finite
                    # error. If that is the case, only the gaussian_fit_error
                    # associated to the finite-error fit will be finite, and so,
                    # we can consider whichever std (p.e. 1.0) to normalize
                    # the gaussian_fit_error since only the finite-error fit
                    # will give a finite error figure-of-merit.
                    gaussian_fit_error = (
                        errors[i][peaks_to_detect][bins_no]['gaussian_fit_error'] - \
                        gaussian_fit_error_mean[i]
                        )/(gaussian_fit_error_std[i] if not fGaussianFitErrorStdIsZero else 1.0)
                    
                # Happens if a cuex.RequestedPeaksNotFound or a
                # RunTime error was triggered by GainMeas.fit_gain()
                # for this (peaks_to_detect, bins_no) combination
                except KeyError:
                    continue
                
                candidate_error_fom = \
                    gaussian_fit_error if not fConsiderLinearFitError else \
                    linear_fit_error + gaussian_fit_error

                try:
                    if candidate_error_fom < optimal_parameters[i]['error_fom']:
                        optimal_parameters[i]['error_fom'] = candidate_error_fom
                        optimal_parameters[i]['peaks_to_detect'] = peaks_to_detect
                        optimal_parameters[i]['bins_no'] = bins_no

                # Happens if optimal_parameters[i]['error_fom'] was still None
                except TypeError:
                    optimal_parameters[i]['error_fom'] = candidate_error_fom
                    optimal_parameters[i]['peaks_to_detect'] = peaks_to_detect
                    optimal_parameters[i]['bins_no'] = bins_no

In [ ]:
# Now, for each gain measurement, we are trying a (peaks_to_detect, bins_no)
# combination that we know for sure will work. So there's no need to handle
# exceptions here.

gain_fit_results = {
    i: {
        'gain_popt': None,
        'gain_stderr': None,
        'histogram_popt': None,
        'histogram_pcov': None,
        'figure': None
    } for i in range(len(gainmeas_objects))
}

for i in range(len(gainmeas_objects)):

    try:
        aux_plot_charge_range = None if params['plot_charge_histogram_range'] is None \
            else params['plot_charge_histogram_range'][gainmeas_objects[i].Overvoltage_V]
    except KeyError:
        raise cuex.InsufficientParameters(
            f"No charge histogram range defined for the "
            f"overvoltage {gainmeas_objects[i].Overvoltage_V} V. "
            "Please, check the input parameters file." 
        )

    aux_fig, aux_ax = plt.subplots(1, 2, figsize=(10, 5))

    # In this case, simply plot the charge histogram in linear and
    # logarithmic scales, without gaussian fits nor linear gain-fit
    if analysis_reliability[i] < 2:
        if params['verbose']:
            print(
                f"Skipping the gain computation for the {i+1}-th "
                f"(/{len(gainmeas_objects)}) SiPM data "
                f"({gainmeas_objects[i].get_title()}), which was "
                "tagged as 1-reliable."
            )

            _, _, _ = aux_ax[0].hist(
                gainmeas_objects[i].ChargeEntries,
                bins_no,
                log=False,
                histtype="step")
            
            aux_ax[0].set_xlabel(f"Charge (C)")
            aux_ax[0].set_ylabel(f"Hits")
            aux_ax[0].set_xlim(aux_plot_charge_range)
            aux_ax[0].set_title('Linear charge histogram')
            aux_ax[0].grid()

            _, _, _ = aux_ax[1].hist(
                gainmeas_objects[i].ChargeEntries,
                bins_no,
                log=True,
                histtype="step")
            
            aux_ax[1].set_xlabel(f"Charge (C)")
            aux_ax[1].set_ylabel(f"Hits")
            aux_ax[1].set_xlim(aux_plot_charge_range)
            aux_ax[1].set_title('Logarithmic charge histogram')
            aux_ax[1].grid()
            aux_ax[1].set_ylim(10**-1)

    else:
        if params['verbose']:
            print(
                f"Computing the gain for the {i+1}-th "
                f"(/{len(gainmeas_objects)}) SiPM data "
                f"({gainmeas_objects[i].get_title()}) with peaks_"
                f"to_detect={optimal_parameters[i]['peaks_to_detect']}"
                f" and bins_no={optimal_parameters[i]['bins_no']}."
            )

        gain_popt, gain_stderr, histogram_popt, histogram_pcov = \
            gainmeas_objects[i].fit_gain(
                has_pedestal=True,
                peaks_to_detect=optimal_parameters[i]['peaks_to_detect'],
                peaks_to_fit=None,
                bins_no=optimal_parameters[i]['bins_no'],
                starting_fraction=params['peak_search_starting_fraction'],
                step_fraction=params['peak_search_step_fraction'],
                minimal_prominence_wrt_max=params['minimal_prominence_wrt_max'],
                std_no=params['gaussian_fit_std_no'],
                gain_fit_axes=aux_ax[0],
                errorbars_scaling=10.,
                histogram_fit_axes=aux_ax[1],
                logarithmic_plot=params['logarithmic_charge_histogram'],
                histogram_axes_title=gainmeas_objects[i].get_title(abbreviate=True),
                gaussian_plot_npoints=200,
                plot_charge_range=aux_plot_charge_range,
                plot_histogram_fit=True
        )

        gain_fit_results[i]['gain_popt'] = gain_popt
        gain_fit_results[i]['gain_stderr'] = gain_stderr
        gain_fit_results[i]['histogram_popt'] = histogram_popt
        gain_fit_results[i]['histogram_pcov'] = histogram_pcov

    # Save the figure in any case
    gain_fit_results[i]['figure'] = aux_fig

# 6. Report results of integration and gain-fit

In [ ]:
if params['generate_report']:

    for idx in range(len(gainmeas_objects)):

        if params['verbose']:
            print(
                "Reporting the integrated waveforms for the "
                f"{idx+1}-th (/{len(gainmeas_objects)}) SiPM data "
                f"- {gainmeas_objects[idx].get_title()}"
            )

        report_pdf.add_text(
            f"Iterator {idx}, {gainmeas_objects[idx].get_title()}",
            horizontal_pos_frac=None,
            vertical_pos_frac=0.95,
            max_width_frac=0.9,
            font_size=params['title_font_size'],
            horizontally_center=True
        )

        report_pdf.add_text(
            f"Persistency plot w/ integration limits",
            horizontal_pos_frac=None,
            vertical_pos_frac=0.88,
            max_width_frac=0.9,
            font_size=params['subtitle_font_size'],
            horizontally_center=True
        )

        report_pdf.add_text(
            f"Integrated {len(gainmeas_objects[idx].Waveforms)}"
            " fast frames",
            horizontal_pos_frac=None,
            vertical_pos_frac=0.84,
            max_width_frac=0.9,
            font_size=params['text_font_size'],
            horizontally_center=True
        )

        if analysis_reliability[idx] == 0:
            report_pdf.add_text(
                f"WARNING: The analysis for this GainMeas has "
                "been tagged as 0-reliable because this measurement"
                f" contains less than {params['minimum_number_of_waveforms']}"
                " waveforms",
                horizontal_pos_frac=0.40,
                vertical_pos_frac=0.89,
                max_width_frac=0.6,
                font_size=params['text_font_size']-2,
                font_color=colors.red,
                horizontally_center=False
            )
        elif analysis_reliability[idx] == 1:
            report_pdf.add_text(
                f"WARNING: The analysis for this GainMeas has "
                "been tagged as 1-reliable because the charge "
                f"histogram could not be fit.",
                horizontal_pos_frac=0.40,
                vertical_pos_frac=0.85,
                max_width_frac=0.6,
                font_size=params['text_font_size']-2,
                font_color=colors.red,
                horizontally_center=False
            )

        aux = gainmeas_objects[idx].Waveforms.plot(
            wvfs_to_plot=params['superimposed_waveforms'],
            ylim=tuple(params['persistency_plot_range']) if params['persistency_plot_range'] is not None else None,
            x0=[],
            y0=[],
            subtract_baseline=params['subtract_baseline_in_persistency_plot'],
            randomize=True,
            fig_title=f"Iterator {idx}, {gainmeas_objects[idx].get_title(abbreviate=True)}\n"
            f"({params['superimposed_waveforms']} waveforms)",
            mode='superposition',
            title_fontsize=params['text_font_size'],
            wvf_linewidth=params['persistency_wvf_linewidth'],
            show_plots=False
        )

        report_pdf.add_plot(
            aux[0],
            horizontal_pos_frac=None,
            vertical_pos_frac=0.32,
            plot_width_wrt_page_width=0.75,
            horizontally_center=True
        )

        if analysis_reliability[idx] > 0:
            if params['verbose']:
                print(
                    "Reporting the gain-fit results for the "
                    f"{idx+1}-th (/{len(gainmeas_objects)}) SiPM data "
                    f"- {gainmeas_objects[idx].get_title()}."
                )

            report_pdf.add_text(
                f"Fit-gain results"
                if analysis_reliability[idx] == 2 else
                f"Not fitted charge histogram",
                horizontal_pos_frac=None,
                vertical_pos_frac=0.35,
                max_width_frac=0.9,
                font_size=params['subtitle_font_size'],
                horizontally_center=True
            )

            report_pdf.add_plot(
                gain_fit_results[idx]['figure'],
                horizontal_pos_frac=None,
                vertical_pos_frac=-0.1,
                plot_width_wrt_page_width=0.8,
                horizontally_center=True
            )
        else:
            if params['verbose']:
                print(
                    "Skipping the report of the gain-fit results for "
                    f"the {idx+1}-th (/{len(gainmeas_objects)}) SiPM "
                    f"data ({gainmeas_objects[idx].get_title()}) "
                    "which was tagged as 0-reliable."
                )
    
        started_a_new_pdf, aux = PDFGenerator.smart_close_page(
            report_pdf,
            params['max_pages_per_pdf_chunk'],
            pdf_chunks_filepaths_generator
        )

        if started_a_new_pdf:
            del report_pdf
            report_pdf = aux
            pdf_reports_filepaths.append(report_pdf.OutputFilepath)

# 7. Generate output dataframe

In [ ]:
data = None
fIsFirst = True

for i in range(len(gainmeas_objects)):

    aux = gainmeas_objects[i].output_summary(
        additional_entries= {
            "merged_points": params['merged_points'],
            "signal_fraction_for_median_cutoff": params['signal_fraction_for_median_cutoff'], 
            "rising_edge_tolerance": params['rising_edge_tolerance'],
            "integration_lower_lim_correction_s": params['integration_lower_lim_correction_s'],
            "integration_lim_width_s": params['integration_lim_width_s'],
            "integration_lower_lim_s": integration_lower_lim_s,
            "integration_upper_lim_s": integration_upper_lim_s,
            "input_resistance_in_ohms": params['input_resistance_in_ohms'],
            "system_amplification_factor": params['system_amplification_factor'],
            "peak_search_starting_fraction": params['peak_search_starting_fraction'],
            "peak_search_step_fraction": params['peak_search_step_fraction'],
            "minimal_prominence_wrt_max": params['minimal_prominence_wrt_max'],
            "gaussian_fit_std_no": params['gaussian_fit_std_no'],
            "min_peaks_to_detect": params['min_peaks_to_detect'],
            "max_peaks_to_detect": params['max_peaks_to_detect'],
            "min_bins_no": params['min_bins_no'],
            "max_bins_no": params['max_bins_no'],
            "bins_no_points": params['bins_no_points'],
            "gain_in_#e-": gain_fit_results[i]['gain_popt'][0]/spcon.e 
            if analysis_reliability[i] > 1 else float('nan'),
            "analysis_reliability": analysis_reliability[i]
        },
        folderpath=None,
        include_analysis_results=True if analysis_reliability[i] > 0 else False,
        overwrite=params['json_overwrite'],
        indent=params['indent'],
        verbose=params['verbose'])
    
    if fIsFirst:
        # Convert the model data to a dictionary of lists
        data = {key: [value] for key, value in aux.items()}
        fIsFirst = False
    else:
        for key in data.keys():
            data[key].append(aux[key])

output_dataframe = pd.DataFrame(data)

if params['verbose']:
    print(
        f"Saving the output dataframe to "
        f"{os.path.abspath(summary_dir)}/{params['output_dataframe_filename']}"
    )

output_dataframe.to_csv(
    os.path.join(
        summary_dir,
        params['output_dataframe_filename']+'.csv'
    )
)

output_dataframe.to_pickle(
    os.path.join(
        summary_dir,
        params['output_dataframe_filename']+'.pkl'
    )
)

# 8. Report the gain resulting distributions

In [ ]:
field_to_show = 'gain_in_#e-'

red_lightness = 0.7

overvoltage_grouped_dataframes = {
    key: group for key, group in output_dataframe.groupby('overvoltage_V')
}
gain_characteristics_with_overvoltage = {
    key: {
        'mean': np.mean(overvoltage_grouped_dataframes[key]['gain_in_#e-']),
        'std': np.std(overvoltage_grouped_dataframes[key]['gain_in_#e-'])
    } for key in overvoltage_grouped_dataframes.keys()
}

# Assuming that red_lightness and gain_characteristics_with_overvoltage are defined
def colour_decide(
        gain: float,
        overvoltage_key: float,
        std_number_soft: int,
        std_number_hard: int
        ):
    
    if math.isnan(gain):
        return (0.8, 0.8, 0.)
    elif gain <= gain_characteristics_with_overvoltage[overvoltage_key]['mean'] - \
        (std_number_hard * gain_characteristics_with_overvoltage[overvoltage_key]['std']):
        return (1., 0., 0.)
    elif gain <= gain_characteristics_with_overvoltage[overvoltage_key]['mean'] - \
        (std_number_soft * gain_characteristics_with_overvoltage[overvoltage_key]['std']):
        return (1., red_lightness, red_lightness)
    elif gain <= gain_characteristics_with_overvoltage[overvoltage_key]['mean'] + \
        (std_number_soft * gain_characteristics_with_overvoltage[overvoltage_key]['std']):
        return 'white'
    elif gain <= gain_characteristics_with_overvoltage[overvoltage_key]['mean'] + \
        (std_number_hard * gain_characteristics_with_overvoltage[overvoltage_key]['std']):
        return (1., red_lightness, red_lightness)
    else:
        return (1., 0., 0.)

In [ ]:
if params['generate_report']:

    fig, axes = plt.subplots(
        nrows=1,
        ncols=1,
        figsize=(15, 5)
    )

    aux_min_overvoltage = min(overvoltage_grouped_dataframes.keys())
    aux_max_overvoltage = max(overvoltage_grouped_dataframes.keys())
    gain_histograms_range = (
        gain_characteristics_with_overvoltage[aux_min_overvoltage]['mean'] \
            - (20. * gain_characteristics_with_overvoltage[aux_min_overvoltage]['std']),
        gain_characteristics_with_overvoltage[aux_max_overvoltage]['mean'] \
            + (20. * gain_characteristics_with_overvoltage[aux_max_overvoltage]['std'])
    )

    tools.plot_histogram(
        axes,
        *[
            np.array(overvoltage_grouped_dataframes[overvoltage][field_to_show])
            for overvoltage in overvoltage_grouped_dataframes.keys()
        ],
        bins=params['resulting_distributions_nbins'],
        hist_range=gain_histograms_range,
        density=False,
        xlabel='Gain (# e-)',
        ylabel='Hits',
        legend_labels=[
            f"OV = {overvoltage} V, "
            r"$\mu = $"
            f"{tools.scientific_notation_str(gain_characteristics_with_overvoltage[overvoltage]['mean'], ndigits=params['gain_mean_ndigits'])}, "
            r"$\sigma = $"
            f"{tools.scientific_notation_str(gain_characteristics_with_overvoltage[overvoltage]['std'], ndigits=params['gain_std_ndigits'])}, "
            for overvoltage in overvoltage_grouped_dataframes.keys()
        ],
        figtitle=f"Overvoltage-wise gain distributions",
        fontsize=params['text_font_size'],
        colourful=True,
    )

    if params['plot_vertical_red_threshold']:
        for overvoltage in overvoltage_grouped_dataframes.keys():
            for factor in (-1., +1.):

                axes.axvline(
                    x=gain_characteristics_with_overvoltage[overvoltage]['mean'] + \
                        (factor * params['std_number_soft'] * gain_characteristics_with_overvoltage[overvoltage]['std']),
                    color=(1., red_lightness, red_lightness),
                    linestyle='-',
                    linewidth=params['vertical_thresholds_linewidth']
                )

                axes.axvline(
                    x=gain_characteristics_with_overvoltage[overvoltage]['mean'] + \
                        (factor * params['std_number_hard'] * gain_characteristics_with_overvoltage[overvoltage]['std']),
                    color=(1., 0., 0.),
                    linestyle='-',
                    linewidth=params['vertical_thresholds_linewidth']
                )

    if params['verbose']:
        print(f"Reporting the gain distributions including all of the SiPMs")

    report_pdf.add_text(
        f"Gain resulting distributions",
        horizontal_pos_frac=None,
        vertical_pos_frac=0.95,
        max_width_frac=0.9,
        font_size=params['title_font_size'],
        horizontally_center=True
    )

    fig.tight_layout()
    report_pdf.add_plot(
        fig,
        horizontal_pos_frac=None,
        vertical_pos_frac=0.45,
        plot_width_wrt_page_width=0.99,
        horizontally_center=True
    )

    # Don't close the last page nor save the PDF
    # yet, because we will place some tables in it

# 9. Report the gain result-tables

In [ ]:
# Assuming that there are only
# three different overvoltages
tables_vertical_pos_frac = (
    0.3, 0.2, 0.1
)

if params['generate_report']:
    
    for i, overvoltage in enumerate(overvoltage_grouped_dataframes.keys()):

        table = tools.strip_ID_vs_sipm_location_dataframe(
            overvoltage_grouped_dataframes[overvoltage], 
            field_to_show, 
            significant_figures=params['gain_mean_ndigits']
        )

        fig, ax = plt.subplots(figsize=(15, 2))
        ax.axis('off')
        ax.table(
            cellText=table.values, 
            colLabels=table.columns, 
            rowLabels=[' '+str(aux)+' ' for aux in range(1,7)], 
            colWidths = [0.06 for _ in table.columns],
            cellColours = [
                [
                    colour_decide(
                        float(val),
                        overvoltage,
                        params['std_number_soft'],
                        params['std_number_hard']
                    ) for val in row
                ] for row in table.values
            ],
            cellLoc = 'center',
            loc='center'
        )
        ax.set_title(
            f"{field_to_show} @ OV = {overvoltage} V"
        )
        
        report_pdf.add_plot(
            fig,
            horizontal_pos_frac=None,
            vertical_pos_frac=tables_vertical_pos_frac[i],
            plot_width_wrt_page_width=0.99,
            horizontally_center=True
        )

    report_pdf.save()

# 10. Generate the cover

In [ ]:
if params['generate_report']:

    path_to_cover_pdf = os.path.join(
        summary_dir,
        'temp_cover.pdf'
    )

    cover_pdf = PDFGenerator(path_to_cover_pdf)

    cover_pdf.add_text(
        f"Gain analysis report",
        horizontal_pos_frac=None,
        vertical_pos_frac=0.95,
        max_width_frac=0.9,
        font_size=params['title_font_size'],
        horizontally_center=True
    )

    clustered_boards_string = tools.get_string_of_contiguously_clustered_integers(
        tools.cluster_integers_by_contiguity(list(output_dataframe.groupby('strip_ID').groups.keys()))
    )

    cover_pdf.add_text(
        f"Analyzed boards: {clustered_boards_string}",
        horizontal_pos_frac=None,
        vertical_pos_frac=0.90,
        max_width_frac=0.9,
        font_size=params['subtitle_font_size'],
        horizontally_center=True
    )

    cover_pdf.add_text(
        f"The results have been dumped to both, a CSV file "
        f"('{params['output_dataframe_filename']}.csv') and a pickle file "
        f"('{params['output_dataframe_filename']}.pkl'), saved alongside this"
        " report.", 
        horizontal_pos_frac=None,
        vertical_pos_frac=0.80,
        max_width_frac=0.9,
        font_size=params['text_font_size'],
        horizontally_center=True
    )

    # n_reliable_analyses[i] gives the number of SiPMs
    # with analysis_reliability[i] equal to n, where
    # n = 0, 1
    n_reliable_analyses = {
        i: np.count_nonzero(np.array(analysis_reliability) == i)
        for i in range(2)
    }

    if sum(list(n_reliable_analyses.values())) == 0:
        cover_pdf.add_text(
            f"All of the analyses have been tagged as reliable",
            horizontal_pos_frac=None,
            vertical_pos_frac=0.71,
            max_width_frac=0.9,
            font_size=params['text_font_size'],
            font_color=colors.green,
            horizontally_center=True
        )

    else:
        cover_pdf.add_text(
            f"A total of {n_reliable_analyses[0]} (resp. {n_reliable_analyses[1]}) analyses,"
            f" out of {len(gainmeas_objects)} - the {round(100.*n_reliable_analyses[0]/len(gainmeas_objects), ndigits=2)} %"
            f"(resp. {round(100.*n_reliable_analyses[1]/len(gainmeas_objects), ndigits=2)} %)- "
            "have been tagged as 0- (resp. 1-) reliable:",
            horizontal_pos_frac=None,
            vertical_pos_frac=0.71,
            max_width_frac=0.9,
            font_size=params['text_font_size'],
            font_color=colors.red,
            horizontally_center=True
        )

        added_lines_so_far = tools.natural_numbers_generator()
        added_lines_so_far_in_this_page = 0
        fNewPage = False
        aux_starting_vertical_pos_frac = {
            True: 0.9, False: 0.67
        }

        for i in range(len(analysis_reliability)):
            if analysis_reliability[i] < 2:
                vertical_pos_frac = \
                    aux_starting_vertical_pos_frac[fNewPage] \
                        - (0.02 * (added_lines_so_far_in_this_page))

                if vertical_pos_frac < 0.1:
                    # Most of the times, the cover PDF will encompass
                    # just one page. For very ill-formed cases, it can
                    # go up to some pages (less than 10). That's why
                    # I am not using the smart-close-page method here.
                    cover_pdf.close_page_and_start_a_new_one()
                    fNewPage = True

                    added_lines_so_far_in_this_page = 0
                    vertical_pos_frac = \
                        aux_starting_vertical_pos_frac[fNewPage]

                cover_pdf.add_text(
                    f"{next(added_lines_so_far)}) {gainmeas_objects[i].get_title()}",
                    horizontal_pos_frac=None,
                    vertical_pos_frac=vertical_pos_frac,
                    max_width_frac=0.9,
                    font_size=params['text_font_size']-4.,
                    font_color={
                        0: colors.red,
                        1: colors.orange
                    }[analysis_reliability[i]],
                    horizontally_center=True
                )

                added_lines_so_far_in_this_page += 1

    cover_pdf.save()

# 11. Join the PDF chunks and add the cover

In [ ]:
if params['generate_report']:

    aux_path_to_report_pdf = os.path.join(
        summary_dir,
        params['report_output_filename']
    )

    if params['verbose']:
        print(
            f"Saving the report PDF to "
            f"{os.path.abspath(aux_path_to_report_pdf)}"
        )

    PDFGenerator.concatenate_PDFs(
        [path_to_cover_pdf]+pdf_reports_filepaths,
        aux_path_to_report_pdf
    )

    os.remove(path_to_cover_pdf)    
    for filepath in pdf_reports_filepaths:
        os.remove(filepath)